## **Erkennung einer optimalen Value-Funktion für GridWorld**

### **GridWorld**

Die GridWorld beschreibt ein 2D-Array. Innerhalb der GridWorld werden ein Start- und ein Zielpunkt definiert. Neben dem  Start- und Zielpunkt kann (muss aber nicht) die GridWorld Hindernisse in Form von Blockformationen beinhalten. Das Ziel soll es nun sein mit Hilfe von Reinforcement Learning einen möglichst schnellen Weg von Start nach Ziel, durch die GridWorld zu finden.

### **Value-Funktionen**

Eine Value-Funktion bestimmt bzw. beschreibt *wie gut* der aktuell verwendete Agent auf Basis seines aktuellen Standpunktes innerhalb der Umgebung mit Inbetrachtnahme seiner Entscheidungen performt. 

Innerhalb der GridWorld muss der Agent immer eine bestimmte Anzahl an Schritten durchführen, um zu seinem Ziel zu gelangen. Dies kann entweder sehr gut funktionieren (der Agent wählt immer einen passenden Schritt und kommt seinem Ziel Schritt für Schritt näher) oder auch sehr schlecht funktionieren, indem der Agent falsche Wege nimmt oder sich z.B. im Kreis dreht (Deadlock).

Es ist also sinnvoll zu bestimmen ob durchgeführte Aktionen des Agenten auf Basis der Value-Funktion gut oder eher schlecht waren. 

Um prüfen zu können, ob die Value-Funktion optimal ist, wird ein Array angelegt mit dem kürzesten Weg (wenigste Schritte) vom aktuellen Index hin zum Ziel:

In [6]:
stepcounts = ([
[ 5, 4, 3, 2, 1,-2, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11],
[ 6, 5, 4, 3, 2, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12],
[ 7, 6,-1, 4, 3, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13],
[ 8, 7,-1, 5,-1,-1,-1,-1,-1, 7,-1,-1,-1,11,12,13,14],
[ 9, 8,-1, 6, 7, 8, 9,10, 9, 8, 9,10,11,12,13,14,15],
[10, 9,-1, 7, 8, 9,10,11,10, 9,10,11,12,13,14,15,16],
[11,10,-1, 8, 9,10,11,12,11,10,11,12,13,14,15,16,17],
[12,11,-1, 9,10,11,12,13,12,11,12,13,14,15,16,17,18],
[13,12,-1,10,11,12,13,14,13,12,13,14,15,16,17,18,19],
[14,13,12,11,12,13,14,15,14,13,14,15,16,17,18,19,20],
[15,14,13,12,13,14,15,16,15,14,15,16,17,18,19,20,21]
])

**-2** definiert das Ziel, **-1** Hindernise und die **sonstigen Zahlen** die Anzahl der Schritte, die benötigt werden um auf schnellsten Wege zum Ziel zu kommen. 

Nachfolgende Funktion berechnet nun die Anzahl der Schritte der aktuell verwendeten Value-Funktion des Agents:

In [1]:
import numpy as np

def check_optimal(self, sv):
    result = np.zeros((self.environent.gw_y_count, self.environent.gw_x_count))  # Array mit berechneten Steps zum Ziel
    for row in range(len(sv)):  # Schleife durch alle bereits berechneten Statevalues
        for col in range(len(sv[row])):
            crow = row      # Zwischenvariable der aktuellen Row
            ccol = col      # Zwischenvariable der aktuellen Col
            goal = False    # Zum pruefen ob das Zeil erreicht wurde
            dead = False    # Zum pruefen ob es sich aktuell um ein Hindernis oder ein Deadlock handelt
            steps = 0       # Zaehlvariable der angewendeten Schritte vom aktuellen Index aus zum Ziel
            while not goal and not dead:
                # Pruefen ob der aktuelle Index ein Hindernis ist, wenn ja, dann dead
                if bool(self.agent.stateinfos.get(self.environent.coord2state(row, col))):
                    # Aktueller State des Index innerhalb der Statevalues des Agents
                    state = self.agent.stateinfos.get(self.environent.coord2state(crow, ccol))
                    # Die vom Agent gewaehlte Aktion auf Basis der 'Greedy-Q-Policy'
                    action = state.policy_greedy_q_based()
                    # Aktuelle Actionstates des zur Zeit ausgeweahlten States
                    values = state.actionvalues
                    # Erhoehung des Stepcounters
                    steps += 1
                    # Pruefung ob der aktuelle State das Ziel ist oder nicht (wenn nur 0, dann Ziel)
                    if np.all(values):
                        # Erhoehung der Col, wenn 'rechts (0)' vom Agent ausgewaehlt wurde
                        if action == 0:
                            ccol += 1
                        # Erhoehung der Row, wenn 'runter (1)' vom Agent ausgewaehlt wurde
                        elif action == 1:
                            crow += 1
                        # Dezimierung der Col, wenn 'links (2)' vom Agent ausgewaehlt wurde
                        elif action == 2:
                            ccol -= 1
                        # Dezimierung der Row, wenn 'rauf (3)' vom Agent ausgewaehlt wurde
                        elif action == 3:
                            crow -= 1
                        # Ueberschreibung des States mit den neuen Koordinaten
                        state = self.agent.stateinfos.get(self.environent.coord2state(crow, ccol))                        
                    else:
                        goal = True
                    # Wenn die Steps zu groß werden, befindet sich der Agent in einem Deadlock und muss abbrechen
                    if steps > 21:
                        dead = True                        
                else:
                    dead = True
            # Wenn dead, dann Index Value = -1
            if dead:
                result[row][col] = -1
            else:
                result[row][col] = steps
            print("row: {} | col: {} | needs {} steps".format(row, col, steps))
    print(result)

Result:

In [14]:
stepcounts = [ 
 [ 5,  4,  3,  2,  1, -2,  1,  2,  3,  4,  5,  6,  7,  8, -1, -1, -1],
 [ 6,  5,  4,  3,  2,  1,  2,  3,  4,  5,  6,  7, 10, -1, -1, -1, -1],
 [ 7,  6, -1,  4,  3,  2,  3,  4,  5,  6,  7,  8,  9, 10, -1, -1, -1],
 [ 8,  7, -1,  5, -1, -1, -1, -1, -1,  7, -1, -1, -1, -1, -1, -1, -1],
 [ 9,  8, -1,  6,  7,  8,  9, 10,  9,  8,  9, 10, -1, -1, -1, -1, -1],
 [10,  9, -1,  7,  8,  9, 10, 11, 10,  9, 10, 11, -1, -1, -1, -1, -1],
 [11, 10, -1,  8,  9, 10, 11, 12, 11, 10, 11, -1, -1, -1, -1, -1, -1],
 [12, 11, -1,  9, 10, 11, 12, 13, 12, 11, -1, -1, -1, -1, -1, -1, -1]
]

Viele der Schritte sind schon sehr nah am optimalen Wert dran. Allerdings gibt es gerade im unteren rechten Bereich noch sehr viele **-1** Werte. Das liegt daran, dass aufgrund der 'Greedy-Q-Policy' nicht jedes Gebiet ausgiebig erforscht wird. Würde man den Agenten weitere Episoden durchlaufen lassen, würden auch hier akurate Schrittzahlen erreicht werden können.
Aus diesem Grund entstehen hier einige Deadlocks, von denen aus der Agent nicht zu seinem Ziel finden kann.